# Neuromap Project Neurohackademy 2019

In [1]:
# install libraries
import pandas as pd
import folium
import geopy

In [13]:
#load in data 
df_data = pd.read_csv('NeuroMap-38responses.csv', encoding='latin-1')

#view data
df_data.head()

#Get the column with hometown location
locs = df_data.iloc[:,3]

#Print locs - check for typos, etc
locs


0                 Split, Croatia
1          Verona, Veneto, Italy
2    Northampton, United Kingdom
3                      Hong Kong
4               Chicago, IL, USA
5     Wichita, KS, United States
6        Buenos Aires, Argentina
7                    Perm/Russia
8                  Mumbai, India
9               Cincinnati, Ohio
Name: Where were you born (city, state/region, country)?, dtype: object

In [4]:
#
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="neuromap")

#Loop through rows to get the longitude and latitude of the hometowns
lat=[]
long=[]
for location in df_data.iloc[:, 3]:
    location_1 = geolocator.geocode(location, timeout=10)
    #print((location_1.latitude, location_1.longitude))
    
    lat.append(location_1.latitude)
    long.append(location_1.longitude)

#Saving lat and long in separate columns in the dataframe    
df_data['HometownLatitude']=lat
df_data['HometownLatitude'].astype('float')

df_data['HometownLongitude']=long
df_data['HometownLongitude'].astype('float')


'Verona, Veneto, Italy'

In [5]:
#Try to plot the locations on the map!
m = folium.Map()

#(location=[location_1.latitude, location_1.longitude])

#Loop through locations and add the markers on the map
for location in range(len(locs)): 
    folium.Marker([lat[location], long[location]], popup=locs[location]).add_to(m)

m


In [ ]:
def generateBaseMap(default_location=[-34.4243941, 150.89385], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [ ]:
base_map

In [ ]:
m= folium.Map(default_location=[-34.4243941, 150.89385])

In [ ]:
m = folium.Map(
    location=[-34.4243941, 150.89385],
    zoom_start=13
)

folium.Circle(
    radius=100,
    location=[-34.4243941, 150.89385],
    popup='The Waterfront',
    color='crimson',
    fill=False,
).add_to(m)

folium.CircleMarker(
    location=[-34.4243941, 150.89385],
    radius=50,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)


m